In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from modules.get_conn import get_conn
from modules.helper_funcs import check_passing_wo, get_failing_test

In [2]:
conn = get_conn(server='antdw-sqlserver-dev.database.windows.net',
                username='antdwdev_admin',
                password='sm@rtf@ctory123',
                db_name='antdw-dev')

### HNULL DATA

dbo.hnull_workorder
- Work order table for HNULL. Primary Key: Order-Batch Number-Time. 
- The test rig used for D026-001D is inferred from the Manufacturer Number Text column. <br>Those tested in H106 rig has a value of D026-001D H106_T156771 while E102 rig has D026-001D E102_T156771

In [3]:
hnull_wo = pd.read_sql("""SELECT * FROM [dbo].[hnull_workorder] 
                       WHERE [Manufacturer Number Text] IN ('D026-001D H106_T156771', 'D026-001D E102_T156771')   
                       ORDER BY [Time] """, conn)

In [4]:
hnull_wo

,Order,Batch Number,Time,Product,Manufacturer Number Text,Remark,File
0,04904760,B99083,2022-01-06 15:55:00,D026-001D,D026-001D H106_T156771,WI005527,D026-001D H106_T156771(B99083)01 06 2022 15 55...
1,04904760,B99083,2022-01-06 16:11:00,D026-001D,D026-001D H106_T156771,WI005527,D026-001D H106_T156771(B99083)01 06 2022 16 11...
2,04904761,B99084,2022-01-06 16:22:00,D026-001D,D026-001D H106_T156771,WI005527,D026-001D H106_T156771(B99084)01 06 2022 16 22...
3,04904761,B99084,2022-01-06 17:11:00,D026-001D,D026-001D H106_T156771,WI005527,D026-001D H106_T156771(B99084)01 06 2022 17 11...
4,04904757,B99080,2022-01-06 17:25:00,D026-001D,D026-001D H106_T156771,WI005527,D026-001D H106_T156771(B99080)01 06 2022 17 25...
...,...,...,...,...,...,...,...
32002,05206603,B108955,2023-06-27 06:22:00,D026-001D,D026-001D H106_T156771,WI005527,D026-001D H106_T156771(B108955)06 27 2023 06 2...
32003,05206603,B108955,2023-06-27 06:30:00,D026-001D,D026-001D H106_T156771,WI005527,D026-001D H106_T156771(B108955)06 27 2023 06 3...
32004,05206603,B108955,2023-06-27 07:18:00,D026-001D,D026-001D H106_T156771,WI005527,D026-001D H106_T156771(B108955)06 27 2023 07 1...
32005,05206602,B108954,2023-06-27 07:34:00,D026-001D,D026-001D H106_T156771,WI005527,D026-001D H106_T156771(B108954)06 27 2023 07 3...


* dbo.hnull_result - Table containing results for HNULL tests. One row corresponds to 1 HNULL test. 

In [5]:
hnull_result = pd.read_sql("""SELECT TOP(5) * FROM [dbo].[hnull_result]  """, conn)

In [6]:
hnull_result

,Order,Batch Number,Time,Characteristic Number,Characteristic Description,Characteristic Type,Characteristics Class,Decimal Places,Nominal Value,Lower Specification Limit,Upper Specification Limit,Unit,Subgroup Size,Subgroup Type,Measured Value,Attribute,Text,Gage Number
0,05197820,B2740,2023-05-30 14:56:00,Null Plot,Null land C-A,0,3,3,7.112,,7.112,um,2,1,5.080,0,,373
1,05197820,B2740,2023-05-30 14:56:00,Expanded Flow Plot,Threshold result,0,3,3,0.000,,,cis,2,1,0.000,0,,373
2,05197820,B2740,2023-05-30 14:56:00,Expanded Flow Plot,Null Bias Results,0,3,3,0.000,,,mA,2,1,0.000,0,,373
3,05197820,B2740,2023-05-30 14:56:00,Pressure Plot,Null Pressure,0,3,3,1400.000,1000.000,1800.000,psi,2,1,1922.975,0,,373
4,05197820,B2740,2023-05-30 14:56:00,Pressure Plot,Null Bias,0,3,3,0.000,,,mA,2,1,0.000,0,,373


#### Getting HNULL Data

To get the HNULL test results, you need to join hnull workorder table and hnull result table on the primary key (Order-Batch Number-Time)

In [ ]:
wo_result_script = """
SELECT  
    WO.[Order], 
    WO.[Batch Number], 
    WO.[Time], 
    WO.[Manufacturer Number Text],
    RES.[Characteristic Description],
    RES.[Measured Value],
    RES.[Lower Specification Limit],
    RES.[Upper Specification Limit]
FROM [dbo].[hnull_workorder] WO 
    LEFT JOIN [dbo].[hnull_result] RES on 
    WO.[Order] = RES.[Order] AND 
    WO.[Batch Number] = RES.[Batch Number] AND 
    WO.[Time] = RES.[Time]
WHERE [Manufacturer Number Text] 
    IN ('D026-001D H106_T156771', 'D026-001D E102_T156771')
"""

hnull_wo_result = pd.read_sql(wo_result_script, conn)

In [ ]:
# Removing workorders and tests with no measured values
hnull_wo_result = hnull_wo_result.dropna(
    subset=['Order', 'Batch Number', 'Time', 'Characteristic Description', 'Measured Value']).drop_duplicates()
hnull_wo_result = hnull_wo_result[~((hnull_wo_result['Batch Number'] == '') | (hnull_wo_result['Order'] == ''))]

In [ ]:
numeric_cols = ['Measured Value', 'Lower Specification Limit', 'Upper Specification Limit']
hnull_wo_result[numeric_cols] = hnull_wo_result[numeric_cols].replace('', np.nan).astype(float)

In [ ]:
hnull_wo_result = hnull_wo_result.drop_duplicates(
    subset=['Order', 'Batch Number', 'Time', 'Characteristic Description', 'Measured Value'])

In [ ]:
hnull_feature_set = hnull_wo_result.pivot(index=['Order', 'Batch Number', 'Time'],
                                          columns='Characteristic Description',
                                          values="Measured Value")

In [ ]:
# Hnull cleaning
# % of null values per column
(hnull_feature_set.isnull().sum() / hnull_feature_set.shape[0]) * 100

In [ ]:
### Dropping columns (i.e. hnull tests) with at least 50% of it's values missing
hnull_feature_set_filtered = hnull_feature_set.loc[:, hnull_feature_set.isin(['',np.nan]).mean() < .5]
hnull_feature_set_filtered = hnull_feature_set_filtered.astype(float)

In [ ]:
hnull_feature_set_filtered.hist(figsize=(20,12));
plt.show()

In [ ]:
hnull_feature_set_filtered = hnull_feature_set_filtered.loc[:,hnull_feature_set_filtered.nunique()>1]

In [ ]:
hnull_feature_set_filtered = hnull_feature_set_filtered.dropna()

In [ ]:
hnull_feature_set_filtered = hnull_feature_set_filtered.merge(check_passing_wo(hnull_wo_result, process='HNULL'), left_index=True, right_index=True)

In [ ]:
test_cols = ['Null Pressure','Pressure Gain', 'Peak Leakage']
hnull_feature_set_filtered.loc[~(hnull_feature_set_filtered[test_cols]==0).all(axis=1), 'Complete'] = True
hnull_feature_set_filtered['Complete'].fillna(False, inplace=True)

In [ ]:
hnull_feature_set_filtered

### PRE-ATP

dbo.atp_workorder - similar to hnull workorder except its related to all ATP related workorders (PRE-ATP, Final ATP etc.) Interest is on PRE-ATP

In [ ]:
atp_wo = pd.read_sql("""SELECT TOP(10) * FROM [dbo].[atp_workorder] WHERE Product = 'D026-001D PRE-ATP' ORDER BY [Time]""", conn)

In [ ]:
atp_wo.head()

dbo.atp_result - similar to hnull result except its related to all ATP related tests (PRE-ATP, Final ATP etc.). Our Interest is on Null Pressure Test (CPPN) although feel free to explore other tests

In [ ]:
atp_result = pd.read_sql("""SELECT TOP(5) * FROM [dbo].[atp_result]  """, conn)

In [ ]:
# Filtering for CPPN only.
atp_result_cppn = pd.read_sql("""SELECT TOP(5) * FROM [dbo].[atp_result] WHERE [Characteristic Description] = 'Null Pressure_PRESSURE TEST' """, conn)

#### Getting PRE-ATP Data

In [ ]:
get_atp_script = """
SELECT  
    WO.[Order], 
    WO.[Batch Number], 
    WO.[Time], 
    WO.[Product],
    RES.[Characteristic Description],
    RES.[Measured Value],
    RES.[Lower Specification Limit],
    RES.[Upper Specification Limit]
FROM [dbo].[atp_workorder] WO 
    LEFT JOIN [dbo].[atp_result] RES on 
        WO.[Order] = RES.[Order] AND 
        WO.[Batch Number] = RES.[Batch Number] AND 
        WO.[Time] = RES.[Time]
WHERE Product IN ('D026-001D PRE-ATP', 'D026-001D PGAL') """

atp_wo_result = pd.read_sql(get_atp_script, conn)

In [ ]:
atp_wo_result.Order = atp_wo_result.Order.str.replace(".","").str.replace(" ","").str.strip()

In [ ]:
# Removing workorders and tests with no measured values
atp_wo_result = atp_wo_result.dropna(
    subset=['Order', 'Batch Number', 'Time', 'Characteristic Description', 'Measured Value']).drop_duplicates()
atp_wo_result = atp_wo_result[~(
    (atp_wo_result['Batch Number'] == '') | (atp_wo_result['Order'] == ''))]

In [ ]:
atp_wo_result[numeric_cols] = atp_wo_result[numeric_cols].replace('', np.nan).astype(float)

In [ ]:
atp_target = atp_wo_result.pivot(index=['Order', 'Batch Number', 'Time', 'Product'],
                                 columns='Characteristic Description', 
                                 values="Measured Value")

In [ ]:
atp_target = atp_target.merge(check_passing_wo(atp_wo_result, process='ATP'), left_index=True, right_index=True)

In [ ]:
atp_target  = atp_target.merge(get_failing_test(atp_wo_result).reset_index(), how='left', 
                 on=['Order','Batch Number', 'Time'])

In [ ]:
# % of null values per column
(atp_target.isnull().sum() / atp_target.shape[0]) * 100

In [ ]:
### Dropping columns (i.e. hnull tests) with at least 75% of it's values missing
atp_target_filtered = atp_target.loc[:, atp_target.isin(['',np.nan]).mean() < .9]

In [ ]:
test_cols = ['Null Pressure_PRESSURE TEST']
atp_target_filtered.loc[~atp_target_filtered[test_cols].isna().any(axis=1), 'Complete'] = True
atp_target_filtered["Complete"].fillna(False, inplace=True)

### Merging HNULL and PRE-ATP data

We get all valid HNULL, and PATP results. 

A valid HNULL entry is complete and passing. Incomplete and Failing HNULL WO do not go through PATP. In case there are multiple valid HNULL entries with same WO/SN, retain latest.

A valid PATP data should have CPPN tests.


PATP and HNULL are matched based on succeeding time stamps (i.e. an HNULL entry with a given WO, SN will be matched to PATP entry of the same WO/SN with a time stamp that succeeds the HNULL time stamp). 
In the case where multiple PATP succeeds a given HNULL data, only the latest time stamp is considered if all succeeding PATP are does not have a CPPN failure. Else, we match the closest PATP entry with CPPN failure.

In [ ]:
valid_hnulls = hnull_feature_set_filtered[hnull_feature_set_filtered.Pass & hnull_feature_set_filtered.Complete].reset_index()
valid_patp = atp_target_filtered[atp_target_filtered.Complete].reset_index(drop=True)

In [ ]:
valid_hnulls = valid_hnulls.rename(columns={"Time":"HNULL_Time", "Complete":"HNULL_Complete","Pass":"HNULL_Pass"})
valid_patp = valid_patp.rename(columns={"Time":"PATP_Time", "Complete":"PATP_Complete","Pass":"PATP_Pass"})
valid_patp["Batch Number"] = valid_patp["Batch Number"].str.rstrip(" Final").str.strip()

In [ ]:
# Retain latest HNULL iteration
valid_hnulls = valid_hnulls.sort_values(['Order','Batch Number', 'HNULL_Time']).drop_duplicates(['Order','Batch Number'], keep='last')

In [ ]:
# We find which PATP work orders have at least 1 CPPN failure
valid_patp['CPPN_FAIL'] = valid_patp['Test'].apply(lambda x: 'Null Pressure_PRESSURE TEST' in str(x))
workorders_with_cppn = valid_patp.groupby(["Order","Batch Number"])['CPPN_FAIL'].any().reset_index()
workorders_with_cppn = workorders_with_cppn.rename(columns={"CPPN_FAIL":"HAS_ATLEAST_1_CPPN_FAIL"})

In [ ]:
valid_patp = valid_patp.merge(workorders_with_cppn, how='left', on=['Order','Batch Number'])

In [ ]:
patp_cppn = valid_patp[valid_patp['HAS_ATLEAST_1_CPPN_FAIL']]
patp_no_cppn = valid_patp[~valid_patp['HAS_ATLEAST_1_CPPN_FAIL']]

In [ ]:
patp_cppn_valid = (patp_cppn.sort_values(by=['Order','Batch Number', 'CPPN_FAIL', 'PATP_Time'],
                                         ascending=[True, True, False, True])
                   .drop_duplicates(['Order','Batch Number'], keep='first'))
patp_no_cppn_valid = (patp_no_cppn.sort_values(by=['PATP_Time'], ascending=False)
                      .drop_duplicates(['Order','Batch Number'], keep='first'))
valid_patp = pd.concat([patp_cppn_valid, patp_no_cppn_valid])

In [ ]:
merged_df = valid_hnulls.merge(valid_patp, on=["Order","Batch Number"])

In [ ]:
merged_df = merged_df.drop(columns='Test')

In [ ]:
final_df = merged_df[merged_df['HNULL_Time']<merged_df['PATP_Time']]

In [ ]:
final_df